In [2]:
import pandas as pd

city_df = pd.read_csv("city-basics_iowa.csv")
lodes_df = pd.read_csv("ia_od_main_JT00_2010.csv")
lodes_df = lodes_df.drop(['createdate'], axis=1)
lodes_df.columns = ["work block", "home block", "total jobs", "Age [0-29]", "Age [30-54]", "Age [55+]", "Salary, Monthly [0-1250]", "Salary, Monthly [1251-3333]", "Salary, Monthly [3334+]", "Jobs: Goods Producing", "Jobs: Trade, Transport, Utilities", "Jobs: Other"]

lodes_df["wb census"] = lodes_df["work block"].apply(lambda x: str(x)[:12])
lodes_df["hb census"] = lodes_df["home block"].apply(lambda x: str(x)[:12])
lodes_df["year"] = "2010"

In [17]:
blockMappings = pd.read_csv("new-block-city-mappings.csv")

censusDict = blockMappings[["Census Block ID", "City"]].set_index("Census Block ID").to_dict()["City"]
latDict = blockMappings[["City", "Lat"]].set_index("City").to_dict()["Lat"]
lonDict = blockMappings[["City", "Lon"]].set_index("City").to_dict()["Lon"]

latDf = pd.DataFrame.from_dict(latDict, orient="index").reset_index()
lonDf = pd.DataFrame.from_dict(lonDict, orient="index").reset_index()
censusDf = pd.DataFrame.from_dict(censusDict, orient="index").reset_index()

In [19]:
def getCity(censusBlock):
    return censusDict[int(censusBlock)]
#     return df[df["Census Block ID"] == censusBlock]["City"]
    
testSet = lodes_df
testSet = testSet.drop(['work block', 'home block'], axis=1)
testSet = testSet.groupby(['wb census', 'hb census']).sum().reset_index()
testSet["home city"] = testSet["hb census"].apply(getCity)
testSet = testSet.drop("hb census", axis=1)
testSet["work city"] = testSet["wb census"].apply(getCity)
testSet = testSet.drop("wb census", axis=1)
testSet.sort_values(["home city", "work city"], inplace=True)
testSet.set_index(["home city", "work city"], inplace=True)

lodes_df = pd.read_csv("ia_od_main_JT00_2010.csv")
lodes_df = lodes_df.drop(['createdate'], axis=1)
lodes_df.columns = ["work block", "home block", "total jobs", "Age [0-29]", "Age [30-54]", "Age [55+]", "Salary, Monthly [0-1250]", "Salary, Monthly [1251-3333]", "Salary, Monthly [3334+]", "Jobs: Goods Producing", "Jobs: Trade, Transport, Utilities", "Jobs: Other"]

lodes_df["wb census"] = lodes_df["work block"].apply(lambda x: str(x)[:12])
lodes_df["hb census"] = lodes_df["home block"].apply(lambda x: str(x)[:12])
lodes_df["year"] = "2010"

In [20]:
import math
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/Users/ewd/ShrinkSmart/')
from ShrinkSmart import ShrinkSmart

jobFlow = pd.pivot_table(testSet, index=["home city", "work city"], aggfunc=sum).reset_index()

for city in ShrinkSmart.cities:
    jobPlot = jobFlow.loc[jobFlow["home city"] == city].sort_values("total jobs", ascending=0)
    jobPlot["x"] = jobPlot["work city"].apply(lambda x: lonDict[x])
    jobPlot["y"] = jobPlot["work city"].apply(lambda x: latDict[x])
    jobPlot["log jobs"] = jobPlot["total jobs"].apply(lambda x: math.log2(x))

    jobFlow["lon work"] = jobFlow["work city"].apply(lambda x: lonDict[x])
    jobFlow["lat work"] = jobFlow["work city"].apply(lambda x: latDict[x])

In [21]:
featureData = pd.DataFrame(jobFlow)
featureData["lon home"] = featureData["home city"].apply(lambda x: float(lonDf[lonDf["index"] == x][0]))
featureData["lat home"] = featureData["home city"].apply(lambda x: float(latDf[latDf["index"] == x][0]))
featureData.to_csv("iowa-job-out.csv")

In [22]:
from tqdm import tqdm

workSet = pd.DataFrame(featureData)
#workSet = workSet.drop(['work block', 'home block'], axis=1)
#testSet = testSet.groupby(['wb census', 'hb census']).sum().reset_index()
#workSet["home city"] = workSet["hb census"].apply(getCity)
#workSet = workSet.drop("hb census", axis=1)
#workSet["work city"] = workSet["wb census"].apply(getCity)
#workSet = workSet.drop("wb census", axis=1)
workSet.sort_values(["work city", "home city"], inplace=True)
workSet.set_index(["work city", "home city"], inplace=True)
workPivot = pd.pivot_table(workSet, index=["work city", "home city"], aggfunc=sum).reset_index()
#workPlot = workFlow.loc[workFlow["work city"] == "Altoona, IA"].sort_values("total jobs", ascending=0)
#workPivot = workPivot.drop(["x(h)"], axis=1)
workPivot

,work city,home city,Age [0-29],Age [30-54],Age [55+],Jobs: Goods Producing,Jobs: Other,"Jobs: Trade, Transport, Utilities","Salary, Monthly [0-1250]","Salary, Monthly [1251-3333]","Salary, Monthly [3334+]",lat home,lat work,lon home,lon work,total jobs
0,"Ackley, IA","Ackley, IA",21,40,29,2,82,6,44,33,13,42.556675,42.556675,-92.976881,-92.976881,90
1,"Ackley, IA","Alden, IA",0,1,1,0,1,1,0,1,1,42.557563,42.556675,-93.380889,-92.976881,2
2,"Ackley, IA","Allison, IA",0,1,0,0,1,0,0,1,0,42.822041,42.556675,-92.761757,-92.976881,1
3,"Ackley, IA","Ames, IA",1,0,0,0,1,0,0,1,0,42.005631,42.556675,-93.649207,-92.976881,1
4,"Ackley, IA","Anamosa, IA",0,1,0,0,1,0,0,0,1,42.104987,42.556675,-91.278666,-92.976881,1
5,"Ackley, IA","Beaman, IA",0,1,0,0,1,0,0,0,1,42.210067,42.556675,-92.776527,-92.976881,1
6,"Ackley, IA","Belle Plaine, IA",0,1,0,0,0,1,1,0,0,41.899067,42.556675,-92.278236,-92.976881,1
7,"Ackley, IA","Bristow, IA",1,6,2,0,9,0,2,6,1,42.758904,42.556675,-92.936815,-92.976881,9
8,"Ackley, IA","Cedar Falls, IA",2,0,0,0,1,1,1,1,0,42.536881,42.556675,-92.444156,-92.976881,2
9,"Ackley, IA","Charles City, IA",1,0,0,0,1,0,1,0,0,43.083170,42.556675,-92.713044,-92.976881,1


In [23]:
workPivot.to_csv("iowa-work-flow.csv")